In [1]:
# parcel_hyperalignment.py
# runs hyperalignment on each parcel individually, saves common spaces and mappers
import utils
import numpy as np
import os, sys, glob, time
import multiprocessing as mp
from mvpa2.mappers.zscore import zscore
from mvpa2.algorithms.hyperalignment import Hyperalignment
from mvpa2.datasets import Dataset
from mvpa2.base import debug
from datetime import timedelta
from mvpa2.base.hdf5 import h5save, h5load

NPROC = 24
os.environ['TMPDIR']='/dartfs-hpc/scratch/f0040y1'
os.environ['TEMP']='/dartfs-hpc/scratch/f0040y1'
os.environ['TMP']='/dartfs-hpc/scratch/f0040y1'
TEMPORARY_OUTDIR='/dartfs-hpc/scratch/f0040y1'

In [5]:
def get_subject_list(nsubs=None):
    # load files 
    in_train_files = glob.glob(os.path.join(utils.sponpain_by_parcel, '*'))
    in_train_subs = np.unique([os.path.basename(x)[0:13] for x in in_train_files])

    in_test_files = glob.glob(os.path.join(utils.bladderpain_by_parcel, '*'))
    in_test_subs = np.unique([os.path.basename(x)[0:13] for x in in_test_files])

    bothsubs = [sub for sub in in_train_subs if sub in in_test_subs]

    if nsubs is not None:
        bothsubs = bothsubs[:nsubs]
    return bothsubs

In [21]:
def apply_mappers((data_fn, mapper_fn, subject, mapper, parcel_num)):
    data = utils.prep_parcelwise_data(subject, parcel_num, 'bladderpain')
    aligned = zscore((np.asmatrix(data)*mapper._proj).A, chunks_attr=None)
    h5save(data_fn, aligned)
    h5save(mapper_fn, mapper)


In [13]:
parcel_num = 1
# get list of subjects to hyperalign
sub_list = get_subject_list(nsubs=10)

aligned_dirname = os.path.join(utils.common_space_dir, 'parcel_{n:03d}'.format(n=parcel_num))
mapper_dirname = os.path.join(utils.trans_matrices, 'parcel_{n:03d}'.format(n=parcel_num))

for d in [aligned_dirname, mapper_dirname]:
    if not os.path.exists(d):
        os.makedirs(d)

In [14]:
train_dss = [utils.prep_parcelwise_data(sub, parcel_num, 'sponpain') for sub in sub_list]

In [15]:
ha = Hyperalignment(nproc=NPROC, joblib_backend='multiprocessing')

In [16]:
len(train_dss)

10

In [17]:
debug.active += ['HPAL']
t0 = time.time()
ha.train(train_dss)
mappers = ha(train_dss)
t1 = time.time()
print('-------- done training hyperalignment at {B} --------'.format(B=str(timedelta(seconds=t1-t0))))

[HPAL] DBG:                    Hyperalignment Hyperalignment: 3 ca:ca{residual_errors chosen_ref_ds+ training_residual_errors} 12 params:params{joblib_backend=multiprocessing output_dim=None zscore_common=True level2_niter=1...} for 10 datasets
 * Please note: warnings are printed only once, but underlying problem might occur many times *
-------- done training hyperalignment at 0:00:06.956289 --------


In [24]:
print('length of test subjects= '+str(len(train_dss)))
data_fns = [os.path.join(aligned_dirname,'{s}_aligned_dtseries.hdf5'.format(s=s)) for s in sub_list]
mapper_fns = [os.path.join(mapper_dirname,'{s}_trained_mapper.hdf5.gz'.format(s=s)) for s in sub_list]
iterable = zip(data_fns, mapper_fns, sub_list, mappers, np.repeat(parcel_num, len(mappers)))

length of test subjects= 10


In [25]:
apply_mappers(iterable[0])

In [17]:
iterable[0]

('/dartfs-hpc/rc/home/1/f0040y1/CANlab/labdata/projects/OLP4CBP/hyperalignment/parcelwise/data/bladder_raw_in_spon/parcel_001/sub-M80302134_aligned_dtseries.npy',
 '/dartfs-hpc/rc/home/1/f0040y1/CANlab/labdata/projects/OLP4CBP/hyperalignment/parcelwise/transformation_matrices/parcel_001/sub-M80302134_trained_mapper.npz',
 'sub-M80302134',
 ProcrusteanMapper(space='commonspace', oblique_rcond=-1.0),
 1)

In [19]:
for i in range(487):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## Check to see if Output File Counts are Correct

In [1]:
import os, glob
common_space_dir = '/dartfs-hpc/rc/home/1/f0040y1/CANlab/labdata/projects/OLP4CBP/hyperalignment/parcelwise/data/bladder_cleaned_in_spon'
mappers_dir = '/dartfs-hpc/rc/home/1/f0040y1/CANlab/labdata/projects/OLP4CBP/hyperalignment/parcelwise/transformation_matrices'

In [2]:
# look through all commmon space parcel directories and count files, put in list
cs_files = [len(glob.glob(os.path.join(common_space_dir, 'parcel_' + '{:0>3d}'.format(i), '*'))) for i in range(487)]

In [4]:
# check which parcels' jobs didn't finish and need to be re-run
cs_re_run = [i for i, e in enumerate(cs_files) if e != 402]
print(cs_re_run)

[]


In [6]:
len(cs_files)

487

In [7]:
# look through all transformation matrix parcel directories and count files, put in list
tm_files = [len(glob.glob(os.path.join(mappers_dir, 'parcel_' + '{:0>3d}'.format(i), '*'))) for i in range(487)]

In [8]:
# check which parcels' jobs didn't finish and need to be re-run
tm_re_run = [i for i, e in enumerate(tm_files) if e != 402]
print(tm_re_run)

[]


## Check output for GLM files

In [14]:
import os, glob
resdir = '/dartfs-hpc/rc/home/1/f0040y1/CANlab/labdata/projects/OLP4CBP/hyperalignment/parcelwise/results/fl_glm'

In [15]:
# get all folders
subfolders = glob.glob(os.path.join(resdir, '*'))
subfolders

['/dartfs-hpc/rc/home/1/f0040y1/CANlab/labdata/projects/OLP4CBP/hyperalignment/parcelwise/results/fl_glm/M80364787',
 '/dartfs-hpc/rc/home/1/f0040y1/CANlab/labdata/projects/OLP4CBP/hyperalignment/parcelwise/results/fl_glm/M80359226',
 '/dartfs-hpc/rc/home/1/f0040y1/CANlab/labdata/projects/OLP4CBP/hyperalignment/parcelwise/results/fl_glm/M80394681',
 '/dartfs-hpc/rc/home/1/f0040y1/CANlab/labdata/projects/OLP4CBP/hyperalignment/parcelwise/results/fl_glm/M80362938',
 '/dartfs-hpc/rc/home/1/f0040y1/CANlab/labdata/projects/OLP4CBP/hyperalignment/parcelwise/results/fl_glm/M80318592',
 '/dartfs-hpc/rc/home/1/f0040y1/CANlab/labdata/projects/OLP4CBP/hyperalignment/parcelwise/results/fl_glm/M80311855',
 '/dartfs-hpc/rc/home/1/f0040y1/CANlab/labdata/projects/OLP4CBP/hyperalignment/parcelwise/results/fl_glm/M80397077',
 '/dartfs-hpc/rc/home/1/f0040y1/CANlab/labdata/projects/OLP4CBP/hyperalignment/parcelwise/results/fl_glm/M80321756',
 '/dartfs-hpc/rc/home/1/f0040y1/CANlab/labdata/projects/OLP4CBP/

In [16]:
len(subfolders)

175

In [17]:
# get ursis from foldernames
ursis = [res[-9:] for res in subfolders]
ursis

['M80364787',
 'M80359226',
 'M80394681',
 'M80362938',
 'M80318592',
 'M80311855',
 'M80397077',
 'M80321756',
 'M80351906',
 'M80336833',
 'M80396887',
 'M80358716',
 'M80321202',
 'M80326819',
 'M80307015',
 'M80356200',
 'M80334610',
 'M80393391',
 'M80353545',
 'M80302799',
 'M80316833',
 'M80399971',
 'M80321120',
 'M80317076',
 'M80310618',
 'M80399265',
 'M80318052',
 'M80303079',
 'M80375912',
 'M80374686',
 'M80398666',
 'M80325092',
 'M80306684',
 'M80313660',
 'M80341831',
 'M80394858',
 'M80346209',
 'M80344908',
 'M80302134',
 'M80348133',
 'M80390618',
 'M80306284',
 'M80398382',
 'M80372078',
 'M80313658',
 'M80316805',
 'M80318234',
 'M80391633',
 'M80347052',
 'M80347121',
 'M80311434',
 'M80392546',
 'M80354415',
 'M80317101',
 'M80395778',
 'M80336083',
 'M80320091',
 'M80344484',
 'M80336871',
 'M80373165',
 'M80385355',
 'M80340863',
 'M80366570',
 'M80357200',
 'M80399337',
 'M80388445',
 'M80322778',
 'M80331231',
 'M80303260',
 'M80352349',
 'M80395955',
 'M803

In [18]:
# confirm that all acompcor subfolders include a fl model file
my_files = []
for i in range(len(subfolders)):
    thisfile = os.path.join(resdir, ursis[i], 'bladder_pain_aCompCor_antpost', 'sub-'+ursis[i]+'_FL_regressor_individual_model.mat')
    if os.path.isfile(thisfile) == True:
        my_files.append(i)
len(my_files)        

175

In [19]:
# check which ursis are missing
import pandas as pd
glmdir = '/dartfs-hpc/rc/home/1/f0040y1/CANlab/labdata/projects/OLP4CBP/hyperalignment/scripts/parcelwise_ha/glm'
master_ursis = pd.read_csv(os.path.join(glmdir,'univariate_subject_table.csv'))

In [20]:
master_ursis.head(15)

,subids,URSI
0,14,M80374029
1,15,M80359226
2,18,M80313660
3,23,M80338269
4,24,M80309706
5,29,M80341407
6,32,M80317666
7,38,M80341717
8,51,M80388445
9,61,M80316547


In [21]:
master_ursis.shape

(175, 2)

In [22]:
# get missing ursis
master_ursi_list = master_ursis['URSI'].tolist()
missing_ursis = [ursi for ursi in master_ursi_list if ursi not in ursis]
missing_ursis

[]

In [23]:
#get indices for those ursis
missing_indices = [master_ursi_list.index(ind) for ind in missing_ursis]
missing_indices

[]

Note that these indices are all 0 indexed, so when going back to re-run, they should each have 1 added to them to get the correct subjects.